In [7]:
# Load the cleaned dataset
import pandas as pd

cleaned_data_path = "../data/processed/cleaned_dataset.csv"
df = pd.read_csv(cleaned_data_path)

print(f"Dataset loaded with {df.shape[0]} rows and {df.shape[1]} columns.")
df.head()


C:\Users\helin\AppData\Local\Temp\ipykernel_24300\2080166256.py:5: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(cleaned_data_path)


Dataset loaded with 999805 rows and 50 columns.


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


## Inspect Missing Data

Check for missing values in the dataset to decide on imputation or removal strategies.


In [8]:
# Check missing values count and percentage
missing_counts = df.isnull().sum()
missing_percent = (missing_counts / len(df)) * 100
missing_summary = pd.DataFrame({"Missing Count": missing_counts, "Missing %": missing_percent})

print(missing_summary[missing_summary["Missing Count"] > 0].sort_values(by="Missing %", ascending=False))


                    Missing Count  Missing %
WrittenOff                 641638  64.176314
Converted                  641638  64.176314
Rebuilt                    641638  64.176314
NewVehicle                 153050  15.307985
mmcode                        550   0.055011
VehicleType                   550   0.055011
make                          550   0.055011
kilowatts                     550   0.055011
cubiccapacity                 550   0.055011
Cylinders                     550   0.055011
Model                         550   0.055011
NumberOfDoors                 550   0.055011
VehicleIntroDate              550   0.055011
bodytype                      550   0.055011
CapitalOutstanding              2   0.000200


## Handling Missing Data

Based on the missing data analysis, decide to either impute or remove missing values.

- For columns with low missing %, consider imputation.
- For columns with large missing %, consider removal or further investigation.


In [9]:
# Cell 5: Handling Missing Data (fix for CapitalOutstanding conversion)

# Drop columns with large missing percentage
cols_to_drop = ['WrittenOff', 'Converted', 'Rebuilt']
df = df.drop(columns=cols_to_drop)

# Drop rows with missing 'NewVehicle'
df = df.dropna(subset=['NewVehicle'])

# Impute missing values in columns with low missing percentage
low_missing_cols = ['mmcode', 'VehicleType', 'make', 'kilowatts', 'cubiccapacity', 
                    'Cylinders', 'Model', 'NumberOfDoors', 'VehicleIntroDate', 'bodytype']

for col in low_missing_cols:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna(df[col].mode()[0])
    else:
        df[col] = df[col].fillna(df[col].median())

# Fix CapitalOutstanding: convert string with commas to float
df['CapitalOutstanding'] = df['CapitalOutstanding'].astype(str).str.replace(',', '.')
df['CapitalOutstanding'] = pd.to_numeric(df['CapitalOutstanding'], errors='coerce')

# Now fill missing values with median
df['CapitalOutstanding'] = df['CapitalOutstanding'].fillna(df['CapitalOutstanding'].median())

# Check remaining missing data
print(df.isnull().sum())


UnderwrittenCoverID         0
PolicyID                    0
TransactionMonth            0
IsVATRegistered             0
Citizenship                 0
LegalType                   0
Title                       0
Language                    0
Bank                        0
AccountType                 0
MaritalStatus               0
Gender                      0
Country                     0
Province                    0
PostalCode                  0
MainCrestaZone              0
SubCrestaZone               0
ItemType                    0
mmcode                      0
VehicleType                 0
RegistrationYear            0
make                        0
Model                       0
Cylinders                   0
cubiccapacity               0
kilowatts                   0
bodytype                    0
NumberOfDoors               0
VehicleIntroDate            0
CustomValueEstimate         0
AlarmImmobiliser            0
TrackingDevice              0
CapitalOutstanding          0
NewVehicle

## Feature Engineering

Create new features relevant for predicting claims and premiums, such as:

- Policy duration (if start and end dates exist)
- Claim history count or flags
- Vehicle age buckets
- Interaction terms, if applicable


calculate the VehicleAge using the RegistrationYear column and the current year.

In [12]:
import datetime

# Calculate the current year
current_year = datetime.datetime.now().year

# Create the VehicleAge column
df['VehicleAge'] = current_year - df['RegistrationYear']

# Create the VehicleAgeBucket column
df['VehicleAgeBucket'] = pd.cut(
    df['VehicleAge'], 
    bins=[0, 5, 10, 15, 100], 
    labels=['0-5', '6-10', '11-15', '15+']
)

# Verify the new columns
print(df[['VehicleAge', 'VehicleAgeBucket']].head())


   VehicleAge VehicleAgeBucket
0          21              15+
1          21              15+
2          21              15+
3          21              15+
4          21              15+


In [13]:
# Example: Create a binary claim flag
df['HasClaim'] = (df['TotalClaims'] > 0).astype(int)

# Example: Vehicle age buckets
df['VehicleAgeBucket'] = pd.cut(df['VehicleAge'], bins=[0, 5, 10, 15, 100], labels=['0-5', '6-10', '11-15', '15+'])

# Example: Policy duration in days (assuming start_date and end_date columns exist)
if 'PolicyStartDate' in df.columns and 'PolicyEndDate' in df.columns:
    df['PolicyDuration'] = (pd.to_datetime(df['PolicyEndDate']) - pd.to_datetime(df['PolicyStartDate'])).dt.days


In [11]:
print(df.columns)


Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser',
       'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'SumInsured',
       'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected',
       'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product',
       'StatutoryClass', 'StatutoryRiskType', 'TotalPremium', 'TotalClaims',
       'HasClaim'],
      dtype='object')


## Encoding Categorical Variables

Convert categorical columns to numerical format using one-hot encoding or label encoding as appropriate.


In [15]:
from sklearn.preprocessing import LabelEncoder

# For simplicity, use one-hot encoding for all categorical columns with relatively few categories
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Drop columns not needed for modeling or identifiers if necessary
columns_to_drop = ['PolicyID', 'CustomerID'] if 'PolicyID' in df.columns and 'CustomerID' in df.columns else []

df_model = df.drop(columns=columns_to_drop)

# One-hot encode categorical features
df_model = pd.get_dummies(df_model, columns=categorical_cols, drop_first=True)

print(f"Data shape after encoding: {df_model.shape}")


Data shape after encoding: (846755, 798)


## Train-Test Split

Split the dataset into training and testing sets, e.g., 80% training and 20% testing.


In [30]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y) for claim severity prediction (only where claims exist)
df_claims = df_model[df_model['TotalClaims'] > 0]

X = df_claims.drop(columns=['TotalClaims'])
y = df_claims['TotalClaims']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")


Training set size: 1960 samples
Test set size: 490 samples


## Model Building - Claim Severity Prediction

## Subset Data to Policies with Claims

We will only include data where `TotalClaims > 0` for claim severity prediction.


In [22]:
# Subset the dataset to policies with claims
df_claims = df_model[df_model['TotalClaims'] > 0]

X = df_claims.drop(columns=['TotalClaims'])
y = df_claims['TotalClaims']

print(f"Dataset for modeling: {X.shape[0]} rows and {X.shape[1]} features.")


Dataset for modeling: 2450 rows and 797 features.


## Baseline Linear Regression Model

Train a simple Linear Regression model to establish a baseline.


In [19]:
import sklearn
print(sklearn.__version__)


1.7.0


In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Train Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predict on the test set
y_pred_lr = lr_model.predict(X_test)

# Calculate metrics
# Calculate Mean Squared Error
mse_lr = mean_squared_error(y_test, y_pred_lr)
# Calculate RMSE manually
rmse_lr = mse_lr ** 0.5
# Calculate R-squared
r2_lr = r2_score(y_test, y_pred_lr)

# Print results
print(f"Linear Regression MSE: {mse_lr:.4f}")
print(f"Linear Regression RMSE: {rmse_lr:.4f}")
print(f"Linear Regression R-squared: {r2_lr:.4f}")


Linear Regression MSE: 1540828464.1423
Linear Regression RMSE: 39253.3879
Linear Regression R-squared: 0.0387


## Decision Tree Regressor

Train a Decision Tree model for claim severity prediction.


In [25]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Train Decision Tree model
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

# Predict on the test set
y_pred_dt = dt_model.predict(X_test)

# Calculate metrics
# Calculate Mean Squared Error
mse_dt = mean_squared_error(y_test, y_pred_dt)
# Calculate RMSE manually
rmse_dt = mse_dt ** 0.5
# Calculate R-squared
r2_dt = r2_score(y_test, y_pred_dt)

# Print results
print(f"Decision Tree MSE: {mse_dt:.4f}")
print(f"Decision Tree RMSE: {rmse_dt:.4f}")
print(f"Decision Tree R-squared: {r2_dt:.4f}")


Decision Tree MSE: 1874898839.5797
Decision Tree RMSE: 43300.1021
Decision Tree R-squared: -0.1697


## Random Forest Regressor

Train a Random Forest model for claim severity prediction.


In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Train Random Forest model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred_rf = rf_model.predict(X_test)

# Calculate metrics
# Mean Squared Error
mse_rf = mean_squared_error(y_test, y_pred_rf)
# Root Mean Squared Error (RMSE)
rmse_rf = mse_rf ** 0.5
# R-squared
r2_rf = r2_score(y_test, y_pred_rf)

# Print results
print(f"Random Forest MSE: {mse_rf:.4f}")
print(f"Random Forest RMSE: {rmse_rf:.4f}")
print(f"Random Forest R-squared: {r2_rf:.4f}")


Random Forest MSE: 1252690948.0629
Random Forest RMSE: 35393.3744
Random Forest R-squared: 0.2185


## XGBoost Regressor

Train an XGBoost model for claim severity prediction.


In [33]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# Train XGBoost model
xgb_model = XGBRegressor(random_state=42)
xgb_model.fit(X_train, y_train)

# Predict and evaluate
y_pred_xgb = xgb_model.predict(X_test)

# Manually compute RMSE to avoid 'squared' param issue
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = sqrt(mse_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"XGBoost RMSE: {rmse_xgb:.4f}")
print(f"XGBoost R-squared: {r2_xgb:.4f}")


XGBoost RMSE: 39650.7471
XGBoost R-squared: 0.0191


## Hyperparameter Tuning: Random Forest Regressor

Use `GridSearchCV` to find the best combination of hyperparameters for the Random Forest Regressor.


In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# Define hyperparameter grid
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform Grid Search
rf_grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=rf_param_grid,
    cv=3,
    scoring='neg_root_mean_squared_error',
    verbose=2,
    n_jobs=-1
)

# Fit the model
rf_grid_search.fit(X_train, y_train)

# Retrieve the best parameters and best model
rf_best_params = rf_grid_search.best_params_
rf_best_model = rf_grid_search.best_estimator_

print(f"Best Parameters for Random Forest: {rf_best_params}")

# Evaluate the tuned model
y_pred_rf_tuned = rf_best_model.predict(X_test)

# Compute metrics manually to avoid compatibility issues
mse_rf_tuned = mean_squared_error(y_test, y_pred_rf_tuned)
rmse_rf_tuned = sqrt(mse_rf_tuned)  # Manually compute RMSE
r2_rf_tuned = r2_score(y_test, y_pred_rf_tuned)

# Print evaluation metrics
print(f"Tuned Random Forest RMSE: {rmse_rf_tuned:.4f}")
print(f"Tuned Random Forest R-squared: {r2_rf_tuned:.4f}")


Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best Parameters for Random Forest: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}
Tuned Random Forest RMSE: 34070.6446
Tuned Random Forest R-squared: 0.2758


## Hyperparameter Tuning: XGBoost Regressor

Use `GridSearchCV` to find the best combination of hyperparameters for the XGBoost Regressor.


In [35]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# Define hyperparameter grid
xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

# Perform Grid Search
xgb_grid_search = GridSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_grid=xgb_param_grid,
    cv=3,
    scoring='neg_root_mean_squared_error',
    verbose=2,
    n_jobs=-1
)

xgb_grid_search.fit(X_train, y_train)

# Best parameters and model
xgb_best_params = xgb_grid_search.best_params_
xgb_best_model = xgb_grid_search.best_estimator_

print(f"Best Parameters for XGBoost: {xgb_best_params}")

# Evaluate tuned model
y_pred_xgb_tuned = xgb_best_model.predict(X_test)
mse_xgb_tuned = mean_squared_error(y_test, y_pred_xgb_tuned)
rmse_xgb_tuned = sqrt(mse_xgb_tuned)
r2_xgb_tuned = r2_score(y_test, y_pred_xgb_tuned)

print(f"Tuned XGBoost RMSE: {rmse_xgb_tuned:.4f}")
print(f"Tuned XGBoost R-squared: {r2_xgb_tuned:.4f}")


Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best Parameters for XGBoost: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1}
Tuned XGBoost RMSE: 34350.4680
Tuned XGBoost R-squared: 0.2638
